In [26]:
from ape import networks, Contract
import pandas as pd

from helpers.coingecko import get_prices
from helpers.subgraph_endpoints import *
from helpers.aura import get_gravi_in_balancer_pool, weekly_emissions_after_fee, aura_mint_ratio
from helpers.vp_info import get_council_vp_fee, get_voter_vp
from helpers.bribe_info import get_bribe_info, GAUGE_CAP
from helpers.formatter import pct_format, dollar_format

In [ ]:
networks.parse_network_choice("ethereum:mainnet").__enter__()

In [28]:
bal_price, aura_price,  badger_price, rpl_price = get_prices()
data = [["${:.2f}".format(badger_price),  "${:.2f}".format(bal_price), "${:.2f}".format(aura_price), "${:.2f}".format(rpl_price)]]
headers = ["Badger price", "Balancer price", "Aura price", "Rpl price"]
df_prices = pd.DataFrame(data, columns=headers)
df_prices

,Badger price,Balancer price,Aura price,Rpl price
0,$2.70,$5.62,$2.12,$16.72


In [29]:
# NOTE: modify with right proposal (latest & current)
proposal_id_last = "0x965fb50ce65ca8ee973e8d00e896c70e5a78de075b8703bc43079bbf27191301"
proposal_id_current = "0xcbbaf8ec970f2a40fb00a85a02fe539fefb10b2e73f95ce0182451f1dde9bafe"

cost_per_vote_after_fee = get_cost_per_vote_after_fee("bribes-overview-aura")

# check active voter in past proposal as ref
response_last_proposal = get_proposal_info(proposal_id_last)
response_current_proposal = get_proposal_info(proposal_id_current)

block_current_proposal = int(response_current_proposal["snapshot"])
aura_voting_actively_last_round = response_last_proposal["scores_total"]
data = [[block_current_proposal,  f'{aura_voting_actively_last_round:.0f}', dollar_format(cost_per_vote_after_fee)]]
headers = ["Proposal block height", "Past Round Active vlAURA votes", "Cost per Vote Last Round"]
df_llama = pd.DataFrame(data, columns=headers)
df_llama


,Proposal block height,Past Round Active vlAURA votes,Cost per Vote Last Round
0,16037021,8852684,$0.0633


In [30]:
# emissions ecosystem
aura = Contract('0xC0c293ce456fF0ED870ADd98a0828Dd4d2903DBF')
aura_mint_ratio = aura_mint_ratio(aura, block_current_proposal)
weekly_emissions_after_fee = weekly_emissions_after_fee(aura_mint_ratio, bal_price, aura_price)
biweekly_emissions = weekly_emissions_after_fee * 2
data = [[aura_mint_ratio, dollar_format(biweekly_emissions)]]
headers = ["Aura Mint Balancer Ratio", "Bi-weekly Emissions"]
df_emissions = pd.DataFrame(data, columns=headers)
df_emissions

,Aura Mint Balancer Ratio,Bi-weekly Emissions
0,3.650248,$2905479.4688


In [31]:
vlAURA = Contract("0x3Fa73f1E5d8A792C80F426fc8F84FBF7Ce9bBCAC")
vlAURA_ts = vlAURA.totalSupply(block_identifier=block_current_proposal)/1e18
council_fee = get_council_vp_fee(vlAURA, block_current_proposal)

# digg pool
balancer_vault = Contract("0xBA12222222228d8Ba445958a75a0704d566BF2C8")
gravi_in_digg_pool = get_gravi_in_balancer_pool(balancer_vault, block_current_proposal)
total_gravi_vp_for_digg = gravi_in_digg_pool

treasury_votes = get_voter_vp(vlAURA, block_current_proposal)
total_vp_badgerdao = treasury_votes + council_fee

pct_controlled_aura_by_badger_treasury = treasury_votes / vlAURA_ts
pct_controlled_aura_by_badger_treasury_active = treasury_votes / aura_voting_actively_last_round
pct_controlled_aura_by_badger = total_vp_badgerdao/ vlAURA_ts
pct_controlled_aura_by_badger_active = total_vp_badgerdao/ aura_voting_actively_last_round
pct_controlled_aura_by_council_active = total_gravi_vp_for_digg / aura_voting_actively_last_round
data = [[treasury_votes, council_fee, total_vp_badgerdao, gravi_in_digg_pool, total_gravi_vp_for_digg, pct_format(pct_controlled_aura_by_badger), pct_format(pct_controlled_aura_by_badger_active)]]
headers = ["Treasury VP", "Council Fee", "Total Badger VP", "Gravi in Digg pool", "Total Badger Council VP - if voting for DIGG", "Badger Aura Controlled - everyone Votes", "Badger Aura Controlled - proactive voters"]
df_vp = pd.DataFrame(data, columns=headers)
df_vp

,Treasury VP,Council Fee,Total Badger VP,Gravi in Digg pool,Total Badger Council VP - if voting for DIGG,Badger Aura Controlled - everyone Votes,Badger Aura Controlled - proactive voters
0,269987.684476,61804.859476,331792.543952,164896.587674,164896.587674,3.253%,3.748%


In [32]:
vebal = Contract("0xC128a9954e6c874eA3d62ce62B468bA073093F25")
vebal_ts = vebal.totalSupplyAt(block_current_proposal) / 1e18
vebal_aura_controlled = vebal.balanceOfAt("0xaF52695E1bB01A16D33D7194C28C42b10e0Dbec2", block_current_proposal)/1e18
pct_aura_vebal_controlled = vebal_aura_controlled/vebal_ts
pct_badger_vebal_controlled = pct_controlled_aura_by_badger*pct_aura_vebal_controlled
pct_badger_vebal_controlled_active = pct_controlled_aura_by_badger_active*pct_aura_vebal_controlled
pct_council_vebal_controlled_active = pct_controlled_aura_by_council_active * pct_aura_vebal_controlled
data = [[pct_format(pct_aura_vebal_controlled), pct_format(pct_badger_vebal_controlled), pct_format(pct_badger_vebal_controlled_active)]]
headers = ["Aura veBAL controlled", "Badger veBAL controlled - everyone votes", "Badger veBAL controlled - proactive voters"]
df_vebal = pd.DataFrame(data, columns=headers)
df_vebal

,Aura veBAL controlled,Badger veBAL controlled - everyone votes,Badger veBAL controlled - proactive voters
0,23.770%,0.773%,0.891%


In [33]:
# NOTE: block vlaura/vebal relative weight breakdown for badger/wbtc cap
vlAURA_voting_badger_wbtc = treasury_votes
vebal_expected_relative_weight_badger_wbtc = pct_controlled_aura_by_badger_treasury_active * pct_aura_vebal_controlled
data = [[vlAURA_voting_badger_wbtc, pct_format(vebal_expected_relative_weight_badger_wbtc)]]
headers = ["vlAURA badger/wbtc pool", "veBAL relative weight"]
df_bribes_badger_pool = pd.DataFrame(data, columns=headers)
df_bribes_badger_pool

,vlAURA badger/wbtc pool,veBAL relative weight
0,269987.684476,0.725%


In [34]:
# NOTE: block vlaura/vebal relative weight breakdown for digg/wbtc/gravi cap
vlAURA_voting_digg_pool = total_gravi_vp_for_digg
vebal_expected_relative_weight_digg = pct_controlled_aura_by_council_active * pct_aura_vebal_controlled
data = [[vlAURA_voting_digg_pool, pct_format(vebal_expected_relative_weight_digg)]]
headers = ["vlAURA DIGG pool", "veBAL relative weight"]
df_bribes_digg_pool = pd.DataFrame(data, columns=headers)
df_bribes_digg_pool

,vlAURA DIGG pool,veBAL relative weight
0,164896.587674,0.443%


In [35]:
# bribes allocations
bribe_for_reth_badger = 16_000
bribe_for_reth_badger_from_rocket = 225
# since badgerreth pool is core, inspect the prop aura bribe expected from Balancer
tvl_rethbadger = get_tvl_balancer_badgerreth_pool()
badgereth_pool = Contract("0xe340EBfcAA544da8bB1Ee9005F1a346D50Ec422e")
pool_badgereth_supply = badgereth_pool.totalSupply() / 1e18
balancer_protocol_fee_collector = "0xce88686553686DA562CE7Cea497CE749DA109f9F"
fees_balancer_earned = badgereth_pool.balanceOf(balancer_protocol_fee_collector) / 1e18
collected_fee_dollar = (fees_balancer_earned / pool_badgereth_supply) * tvl_rethbadger
bribe_for_reth_badger_core = (
    collected_fee_dollar - collected_fee_dollar * 0.25
) * pct_aura_vebal_controlled
# display
data = [
    [
        bribe_for_reth_badger,
        bribe_for_reth_badger_from_rocket,
        bribe_for_reth_badger_core
    ]
]
headers = [
    "Bribe allocated to badger/reth pool (BADGER)",
    "Bribe allocated to badger/reth pool (RPL) - TBD",
    "Bribe allocated to badger/reth pool (CORE-USDC)"
]
df_bribes_allocation = pd.DataFrame(data, columns=headers)
df_bribes_allocation

,Bribe allocated to badger/reth pool (BADGER),Bribe allocated to badger/reth pool (RPL) - TBD,Bribe allocated to badger/reth pool (CORE-USDC)
0,16000,225,714.652267


In [36]:
# badger/reth prediction for vebal relative weight given the bribe
# NOTE: add in case the rocket confirms co-incentive -> bribe_for_reth_badger_from_rocket * rpl_price 
bribe_badgereth_usd = bribe_for_reth_badger * badger_price + bribe_for_reth_badger_core
vlAURA_votes_bought = bribe_badgereth_usd / cost_per_vote_after_fee
pct_controlled_aura_by_bribing_badger_reth = vlAURA_votes_bought / aura_voting_actively_last_round
vebal_expected_relative_weight_badger_reth = pct_controlled_aura_by_bribing_badger_reth * pct_aura_vebal_controlled
data = [[vlAURA_votes_bought, pct_format(vebal_expected_relative_weight_badger_reth), dollar_format(bribe_badgereth_usd)]]
headers = ["vlAURA badger/reth pool", "veBAL relative weight", "Bribe Cost"]
df_bribe_breakdown_badgereth = pd.DataFrame(data, columns=headers)
df_bribe_breakdown_badgereth

,vlAURA badger/reth pool,veBAL relative weight,Bribe Cost
0,693602.933054,1.862%,$43914.6523


In [37]:
# treasury expected earnings bi-weekly expectation in BADGER/WBTC pool
tvl_badger = get_tvl_balancer_badger_pool()
badger_pool = Contract("0xb460DAa847c45f1C4a41cb05BFB3b51c92e41B36")
pool_supply = badger_pool.totalSupply() / 1e18
reward_badger_aura = Contract("0x05df1E87f41F793D9e03d341Cdc315b76595654C")
vault_rewards_contract_balance = reward_badger_aura.balanceOf("0xD0A7A8B98957b9CD3cFB9c0425AbE44551158e9e", block_identifier=block_current_proposal)/1e18
vault_owned_tcl = tvl_badger * (vault_rewards_contract_balance/pool_supply)
pool_capture = vault_owned_tcl / tvl_badger
pool_earnings = biweekly_emissions * vebal_expected_relative_weight_badger_wbtc
data = [[dollar_format(pool_earnings), dollar_format(tvl_badger), pct_format(pool_capture), dollar_format(pool_earnings*pool_capture)]]
headers = ["Pool yield (USD) - biweekly", "BADGER Pool TVL", "BADGER Pool Capture", "Treasury yield earned (USD) - biweekly"]
df_earnings_badger_pool = pd.DataFrame(data, columns=headers)
df_earnings_badger_pool

,Pool yield (USD) - biweekly,BADGER Pool TVL,BADGER Pool Capture,Treasury yield earned (USD) - biweekly
0,$21062.3887,$5071117.8701,43.934%,$9253.5671


In [38]:
# treasury expected earnings bi-weekly expectation in DIGG pool
tvl_digg = get_tvl_balancer_digg_pool()
digg_pool = Contract("0x8eB6c82C3081bBBd45DcAC5afA631aaC53478b7C")
pool_supply = digg_pool.totalSupply() / 1e18
reward_badger_aura = Contract("0xe86f0312b06126855810b4a13a43c3e2b1b8dd90")
vault_rewards_contract_balance = reward_badger_aura.balanceOf("0xD0A7A8B98957b9CD3cFB9c0425AbE44551158e9e", block_identifier=block_current_proposal)/1e18
vault_owned_tcl = tvl_digg * (vault_rewards_contract_balance/pool_supply)
pool_capture = vault_owned_tcl / tvl_digg
pool_earnings = biweekly_emissions * vebal_expected_relative_weight_digg
data = [[dollar_format(pool_earnings), dollar_format(tvl_digg), pct_format(pool_capture), dollar_format(pool_earnings*pool_capture)]]
headers = ["Pool yield (USD) - biweekly", "DIGG Pool TVL", "DIGG Pool Capture", "Treasury yield earned (USD) - biweekly"]
df_earnings_badger_pool = pd.DataFrame(data, columns=headers)
df_earnings_badger_pool

,Pool yield (USD) - biweekly,DIGG Pool TVL,DIGG Pool Capture,Treasury yield earned (USD) - biweekly
0,$12863.9795,$1804052.2526,41.107%,$5288.0276


In [39]:
# treasury expected earnings bi-weekly expectation post-bribe into BADGER/RETH pool
reward_badger_aura = Contract("0x685c94e7da6c8f14ae58f168c942fb05bad73412")
vault_rewards_contract_balance = reward_badger_aura.balanceOf("0xD0A7A8B98957b9CD3cFB9c0425AbE44551158e9e", block_identifier=block_current_proposal)/1e18
vault_owned_tcl = tvl_digg * (vault_rewards_contract_balance/pool_badgereth_supply)
pool_capture = vault_owned_tcl / tvl_digg
pool_earnings = biweekly_emissions * vebal_expected_relative_weight_badger_reth
# NOTE: remove the core bribe as it is not DAO cost
treasury_roi = (pool_earnings*pool_capture) / (bribe_badgereth_usd - bribe_for_reth_badger_core)
data = [[dollar_format(pool_earnings), dollar_format(tvl_rethbadger), pct_format(pool_capture), dollar_format(pool_earnings*pool_capture), pct_format(treasury_roi)]]
headers = ["Pool yield (USD) - biweekly", "BADGER/RETH Pool TVL", "BADGER/RETH Pool Capture", "Treasury yield earned (USD) - biweekly", "Treasury ROI - biweekly"]
df_earnings_badger_pool = pd.DataFrame(data, columns=headers)
df_earnings_badger_pool


,Pool yield (USD) - biweekly,BADGER/RETH Pool TVL,BADGER/RETH Pool Capture,Treasury yield earned (USD) - biweekly,Treasury ROI - biweekly
0,$54109.6332,$2390018.8826,64.655%,$34984.6582,80.983%


In [40]:
# NOTE: block to max out badger/wbtc cap
vlAURA_req, cost_of_max_cap, badger_to_bribe_badger_pool = get_bribe_info(treasury_votes, pct_controlled_aura_by_badger_treasury_active*pct_aura_vebal_controlled, cost_per_vote_after_fee,badger_price)
bribe_reclycing_badger_voting = badger_to_bribe_badger_pool * (treasury_votes/vlAURA_req)
data = [[vlAURA_req, dollar_format(cost_of_max_cap), badger_to_bribe_badger_pool, bribe_reclycing_badger_voting]]
headers = ["vlAURA Required", "Cost of Maxing Cap", "Badger Bribe Max Cap", "Expected Bribe reclyced"]
df_bribes_badger_pool = pd.DataFrame(data, columns=headers)
df_bribes_badger_pool

,vlAURA Required,Cost of Maxing Cap,Badger Bribe Max Cap,Expected Bribe reclyced
0,744876.26756,$47160.9630,17467.023338,6331.093352


In [41]:
# treasury expected earnings bi-weekly expectation post-bribe into BADGER pool
tvl_badger = get_tvl_balancer_badger_pool()
badger_pool = Contract("0xb460DAa847c45f1C4a41cb05BFB3b51c92e41B36")
pool_supply = badger_pool.totalSupply() / 1e18
reward_badger_aura = Contract("0x05df1E87f41F793D9e03d341Cdc315b76595654C")
vault_rewards_contract_balance = reward_badger_aura.balanceOf("0xD0A7A8B98957b9CD3cFB9c0425AbE44551158e9e", block_identifier=block_current_proposal)/1e18
vault_owned_tcl = tvl_badger * (vault_rewards_contract_balance/pool_supply)
pool_capture = vault_owned_tcl / tvl_badger
pool_earnings = biweekly_emissions * GAUGE_CAP
real_bribe_aura_expense = (badger_to_bribe_badger_pool - bribe_reclycing_badger_voting) * badger_price
treasury_roi = (pool_earnings*pool_capture) / real_bribe_aura_expense
data = [[dollar_format(pool_earnings), dollar_format(tvl_badger), pct_format(pool_capture), dollar_format(pool_earnings*pool_capture), dollar_format(real_bribe_aura_expense), pct_format(treasury_roi)]]
headers = ["Pool yield (USD) - biweekly", "BADGER Pool TVL", "BADGER Pool Capture", "Treasury yield earned (USD) - biweekly", "Bribe Aura Real Cost", "Treasury ROI - biweekly"]
df_earnings_badger_pool = pd.DataFrame(data, columns=headers)
df_earnings_badger_pool

,Pool yield (USD) - biweekly,BADGER Pool TVL,BADGER Pool Capture,Treasury yield earned (USD) - biweekly,Bribe Aura Real Cost,Treasury ROI - biweekly
0,$58109.5894,$5071117.8701,43.934%,$25529.9146,$30067.0110,84.910%
